<a href="https://colab.research.google.com/github/bilzard/JQuants-news/blob/main/004_fetch_latest_stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

この notebook では [JQuantsAPI] をつかって最新の株価データを取得します。

[JQuantsAPI]: https://jpx-jquants.com/apidoc.html


In [197]:
# Google Colab環境ではGoogle Driveをマウントしてアクセスできるようにします。
import sys

if 'google.colab' in sys.modules:
    # Google Drive をマウントします
    from google.colab import drive
    mount_dir = "/content/drive"
    drive.mount(mount_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# データセットを配置したディレクトリのパスを設定
if 'google.colab' in sys.modules:
    dataset_dir = f"{mount_dir}/MyDrive/JPX_competition/data_dir_comp2"
else:
    dataset_dir = "/notebook/data_dir_comp2"

In [ ]:
refresh_token = '(自分のAPI tokenを入力します)'

In [ ]:
import os
import json
import requests
import base64
import pandas as pd
import numpy as np

In [ ]:
def call_jquants_api(params: dict, idtoken: str, apitype: str, code: str = None):
    """
    J-QuantsのAPIを試すメソッド。

    Parameters
    ----------
    params : dict
        リクエストパラメータ。
    idtoken : str
        idTokenはログイン後の画面からご確認いただけます。
    apitype: str
        APIの種類。"news", "prices", "lists"などがあります。
    code: str
        銘柄を指定するAPIの場合に設定します。

    Returns
    -------
    resjson : dict
        APIレスポンス(json形式)
    """
    datefrom = params.get("datefrom", None)
    dateto = params.get("dateto", None)
    date = params.get("date", None)
    includedetails = params.get("includedetails", "false")
    keyword = params.get("keyword", None)
    headline = params.get("headline", None)
    paramcode = params.get("code", None)
    nexttoken = params.get("nextToken", None)
    headers = {"accept": "application/json", "Authorization": idtoken}
    data = {
        "from": datefrom,
        "to": dateto,
        "includeDetails": includedetails,
        "nextToken": nexttoken,
        "date": date,
        "keyword": keyword,
        "headline": headline,
        "code": paramcode,
    }

    if code:
        code = "/" + code
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype + code,
            params=data,
            headers=headers,
        )
    else:
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype, params=data, headers=headers
        )
    resjson = json.loads(r.text)
    return resjson

In [ ]:
class IdToken:
  def __init__(self, refresh_token):
    self._refresh_token = refresh_token
    self.refresh()

  def _get_id_token(self):
    headers = {"accept": "application/json"}
    data = {"refresh-token": self._refresh_token}

    response = requests.post(
        "https://api.jpx-jquants.com/refresh", headers=headers, data=json.dumps(data)
    )

    resjson = json.loads(response.text)
    return resjson

  def refresh(self):
    ret = self._get_id_token()
    self._id_token = ret['idToken']

  def get(self):
    return self._id_token

In [ ]:
class Prices(object):
  def __init__(self, params, id_token):
    self._params = params
    self._id_token = id_token
    self._next_token = None
    self._stop = False

  def __iter__(self):
    return self

  def __next__(self):
    return self.next()

  def next(self):
    if self._stop:
      raise StopIteration()

    params['nextToken'] = self._next_token
    ret = call_jquants_api(params, self._id_token, 'prices')

    if 'nextToken' in ret:
      self._next_token = ret['nextToken']
    else:
      self._next_token = None
      self._stop = True

    return ret['prices']

2021/01/01 - 2021/04/30 までの株価を取得します。
だいたい20〜30分くらいかかります。

In [ ]:
from tqdm import tqdm


idToken = IdToken(refresh_token)
results = []
dates = [dt.strftime('%Y-%m-%d') for dt in pd.date_range(start='2021-01-01', end='2021-04-30')]

for date in tqdm(dates):
  params = {
    'includedetails': 'True',
    'date': date,
    'nextToken': None,
  }
  prices = Prices(params, idToken.get())

  for price in prices:
    results.append(json.dumps(price))

df_price = pd.concat(
    [pd.read_json(json_ret) for json_ret in results],
    ignore_index=True
)

 43%|████▎     | 52/120 [07:30<06:57,  6.14s/it]

In [ ]:
df_price

,EndOfDayQuote Open,EndOfDayQuote PreviousClose,EndOfDayQuote CumulativeAdjustmentFactor,EndOfDayQuote VWAP,EndOfDayQuote Low,EndOfDayQuote PreviousExchangeOfficialClose,EndOfDayQuote High,EndOfDayQuote Date,EndOfDayQuote Close,EndOfDayQuote PreviousExchangeOfficialCloseDate,EndOfDayQuote ExchangeOfficialClose,EndOfDayQuote ChangeFromPreviousClose,EndOfDayQuote PercentChangeFromPreviousClose,EndOfDayQuote PreviousCloseDate,Local Code,EndOfDayQuote Volume
0,2951.0,2951.0,1.0,2932.165,2913.0,2951.0,2951.0,2021/01/04,2950.0,2020/12/30,2950.0,-1.0,-0.034,2020/12/30,1301,9700
1,428.0,426.0,1.0,419.695,416.0,426.0,429.0,2021/01/04,421.0,2020/12/30,421.0,-5.0,-1.174,2020/12/30,1332,1780500
2,2229.0,2220.0,1.0,2200.351,2179.0,2220.0,2231.0,2021/01/04,2202.0,2020/12/30,2202.0,-18.0,-0.811,2020/12/30,1333,112400
3,975.0,988.0,1.0,986.833,975.0,988.0,992.0,2021/01/04,992.0,2020/12/30,992.0,4.0,0.405,2020/12/30,1352,600
4,1701.0,1693.0,1.0,1680.399,1672.0,1693.0,1701.0,2021/01/04,1674.0,2020/12/30,1674.0,-19.0,-1.122,2020/12/30,1375,67900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300445,1722.0,1719.0,1.0,1711.462,1707.0,1719.0,1724.0,2021/04/30,1707.0,2021/04/28,1707.0,-12.0,-0.698,2021/04/28,9993,5200
300446,2336.0,2336.0,1.0,2325.642,2315.0,2336.0,2337.0,2021/04/30,2315.0,2021/04/28,2315.0,-21.0,-0.899,2021/04/28,9994,5300
300447,428.0,430.0,1.0,424.865,421.0,430.0,430.0,2021/04/30,421.0,2021/04/28,421.0,-9.0,-2.093,2021/04/28,9995,83100
300448,1501.0,1502.0,1.0,1496.667,1494.0,1502.0,1501.0,2021/04/30,1494.0,2021/04/28,1494.0,-8.0,-0.533,2021/04/28,9996,300


In [198]:
# ローカルディレクトリに保存
df_price.to_csv('stock_price_latest.csv', index=False)

In [199]:
! ls -l

total 44008
drwx------ 5 root root     4096 May  2 03:55 drive
drwxr-xr-x 1 root root     4096 Apr 21 13:39 sample_data
-rw-r--r-- 1 root root 34741320 May  2 07:27 stock_price_latest.csv
-rw-r--r-- 1 root root 10310270 May  2 04:25 stock_price_latest.csv.gz


In [200]:
# gz ファイルに圧縮
! gzip stock_price_latest.csv.gz stock_price_latest.csv

gzip: stock_price_latest.csv.gz already has .gz suffix -- unchanged
gzip: stock_price_latest.csv.gz already exists; do you wish to overwrite (y or n)? y


In [201]:
! ls -l stock_price_latest.csv.gz

-rw-r--r-- 1 root root 9703414 May  2 07:27 stock_price_latest.csv.gz


In [203]:
! ls -l

total 9484
drwx------ 5 root root    4096 May  2 03:55 drive
drwxr-xr-x 1 root root    4096 Apr 21 13:39 sample_data
-rw-r--r-- 1 root root 9703414 May  2 07:27 stock_price_latest.csv.gz


In [204]:
# Google Drive 上に保存します
! cp stock_price_latest.csv.gz {dataset_dir}/.